In [25]:
import json
import sys
import random

sys.path.append("../")

from models.vector_model import vector_model
from tqdm import tqdm
from preprocessors.preprocessor import Preprocessor

import pandas as pd
import numpy as np

import IR_utils

random.seed(0)
np.random.seed(0)

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
data_path = "../../data/dataset/corpus.jsonl"
docs = IR_utils.load_document_corpus(data_path=data_path, max_docs=10)

print("Number of documents in corpus: {}".format(len(docs)))

Number of documents in corpus: 10


In [28]:
preprocessor = Preprocessor()
tokenized_docs = preprocessor.preprocess(docs)

100%|██████████| 10/10 [00:00<00:00, 2516.53it/s]


In [18]:
vm = vector_model(documents=tokenized_docs, min_df=3)
vm.fit()

print(f"Vocabulary Size: {len(vm.vocab)}")
print(f"Documents Size: {len(vm.docs)}")
print(f"TF-iDF Vectors {vm.vectors.shape}")

10it [00:00, 78840.30it/s]

Vocabulary Size: 19
Documents Size: 10
TF-iDF Vectors (10, 19)


In [24]:
vm.vectors[7]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 2.30258509, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])